# Restaurants recommendation 

In [457]:
import os
import json
import pandas as pd
import numpy as np
from copy import deepcopy


import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from pylab import savefig

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

from keras.models import Sequential, Model, load_model
from keras.utils.vis_utils import model_to_dot
from keras.optimizers import Adam
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Flatten, Dropout, dot
from keras.constraints import non_neg
from IPython.display import SVG

In [458]:
Ratings_TripAdvisor = pd.read_csv('TA_restaurants_curated.csv')
Ratings_TripAdvisor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125527 entries, 0 to 125526
Data columns (total 11 columns):
Unnamed: 0           125527 non-null int64
Name                 125527 non-null object
City                 125527 non-null object
Cuisine Style        94176 non-null object
Ranking              115876 non-null float64
Rating               115897 non-null float64
Price Range          77672 non-null object
Number of Reviews    108183 non-null float64
Reviews              115911 non-null object
URL_TA               125527 non-null object
ID_TA                125527 non-null object
dtypes: float64(3), int64(1), object(7)
memory usage: 10.5+ MB


In [459]:
Ratings_TripAdvisor.head()

,Unnamed: 0,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",/Restaurant_Review-g188590-d11752080-Reviews-M...,d11752080
1,1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",/Restaurant_Review-g188590-d693419-Reviews-De_...,d693419
2,2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567.0,"[['Satisfaction', 'Delicious old school restau...",/Restaurant_Review-g188590-d696959-Reviews-La_...,d696959
3,3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,$$$$,564.0,"[['True five star dinner', 'A superb evening o...",/Restaurant_Review-g188590-d1239229-Reviews-Vi...,d1239229
4,4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,$$$$,316.0,"[['Best meal.... EVER', 'super food experience...",/Restaurant_Review-g188590-d6864170-Reviews-Li...,d6864170


In [460]:
ColumnsDrop = ['URL_TA']
Ratings_v2 = Ratings_TripAdvisor.drop(ColumnsDrop, axis = 1)
Ratings_v2.head()

,Unnamed: 0,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,ID_TA
0,0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",d11752080
1,1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",d693419
2,2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567.0,"[['Satisfaction', 'Delicious old school restau...",d696959
3,3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,$$$$,564.0,"[['True five star dinner', 'A superb evening o...",d1239229
4,4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,$$$$,316.0,"[['Best meal.... EVER', 'super food experience...",d6864170


In [461]:
sum(Ratings_v2['Price Range'].isnull())

47855

In [462]:
Ratings_v3 = Ratings_v2.dropna().drop(['Unnamed: 0'], axis = 1)

In [463]:
Ratings_v3.head()

,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,ID_TA
0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",d11752080
1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",d693419
2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567.0,"[['Satisfaction', 'Delicious old school restau...",d696959
3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,$$$$,564.0,"[['True five star dinner', 'A superb evening o...",d1239229
4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,$$$$,316.0,"[['Best meal.... EVER', 'super food experience...",d6864170


In [464]:
Ratings_v3.rename(columns=lambda x: x.replace(" ", "_"), inplace=True)
Ratings_v3.head()

,Name,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,Reviews,ID_TA
0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",d11752080
1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",d693419
2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567.0,"[['Satisfaction', 'Delicious old school restau...",d696959
3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,$$$$,564.0,"[['True five star dinner', 'A superb evening o...",d1239229
4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,$$$$,316.0,"[['Best meal.... EVER', 'super food experience...",d6864170


In [465]:
Ratings_v3['City'].unique()

array(['Amsterdam', 'Athens', 'Barcelona', 'Berlin', 'Bratislava',
       'Brussels', 'Budapest', 'Copenhagen', 'Dublin', 'Edinburgh',
       'Geneva', 'Hamburg', 'Helsinki', 'Krakow', 'Lisbon', 'Ljubljana',
       'London', 'Luxembourg', 'Lyon', 'Madrid', 'Milan', 'Munich',
       'Oporto', 'Oslo', 'Paris', 'Prague', 'Rome', 'Stockholm', 'Vienna',
       'Warsaw', 'Zurich'], dtype=object)

In [466]:
Ratings_v3['Price_Range'] = Ratings_v3['Price_Range'].apply(lambda x: len(x.split('-')[0].strip()))
Ratings_v3.head()

,Name,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,Reviews,ID_TA
0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,2,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",d11752080
1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,4,812.0,"[['Great food and staff', 'just perfect'], ['0...",d693419
2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,4,567.0,"[['Satisfaction', 'Delicious old school restau...",d696959
3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,4,564.0,"[['True five star dinner', 'A superb evening o...",d1239229
4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,4,316.0,"[['Best meal.... EVER', 'super food experience...",d6864170


In [467]:
len(Ratings_v3), len(Ratings_v3['Name'].unique())

(74225, 67862)

In [468]:
Ratings_v4 = Ratings_v3.drop(['Reviews'], axis = 1)
Ratings_v4.head()

,Name,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,ID_TA
0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,2,136.0,d11752080
1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,4,812.0,d693419
2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,4,567.0,d696959
3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,4,564.0,d1239229
4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,4,316.0,d6864170


In [469]:
Ratings_v4['Cuisine_Style'][0]

"['French', 'Dutch', 'European']"

In [470]:
import re, string
regex = re.compile('[%s]' % re.escape(string.punctuation))
def test_re(s):  # From Vinko's solution, with fix.
    return regex.sub('', s)
#Ratings_v4['Cuisine_Style'].apply(lambda x : print(x[0]) )
Ratings_v4['Cuisine_Style'] = Ratings_v4['Cuisine_Style'].apply(lambda x : [re.sub(r'[^\w\s]','',y.strip()) for y in x.strip().split(',')][0])
#Ratings_v4['Cuisine_Style'][0]

In [471]:
Ratings_v4.head()

,Name,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,ID_TA
0,Martine of Martine's Table,Amsterdam,French,1.0,5.0,2,136.0,d11752080
1,De Silveren Spiegel,Amsterdam,Dutch,2.0,4.5,4,812.0,d693419
2,La Rive,Amsterdam,Mediterranean,3.0,4.5,4,567.0,d696959
3,Vinkeles,Amsterdam,French,4.0,5.0,4,564.0,d1239229
4,Librije's Zusje Amsterdam,Amsterdam,Dutch,5.0,4.5,4,316.0,d6864170


In [472]:
grouped = Ratings_v4.groupby(['City', 'Name'])['Rating'].mean()
grouped2 = grouped.groupby(level = 'City').nlargest(50).reset_index(level= 0, drop = True)

In [473]:
#grouped.groupby(level = 'City').nlargest(50).reset_index(level= 0, drop = True)
grouped3 = grouped2.unstack(level = 0)

In [474]:
grouped2 = pd.DataFrame(grouped2.reset_index())

In [475]:
grouped2.columns
Cities =grouped2['City'].unique()
restaurants = grouped2['Name'].unique()
Ratings_v4[Ratings_v4['Name'].isin(restaurants)]

,Name,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,ID_TA
127,Arles,Amsterdam,French,128.0,5.0,2,87.0,d11671406
141,Bistrot des Alpes,Amsterdam,French,142.0,5.0,2,136.0,d7735788
233,Benny’s Chicken,Amsterdam,Dutch,234.0,5.0,1,88.0,d10522612
359,Cuddle,Amsterdam,Bar,360.0,5.0,1,116.0,d11949427
421,Broer,Amsterdam,Dutch,422.0,5.0,2,50.0,d10525937
475,Chez Lorraine,Amsterdam,French,476.0,5.0,2,46.0,d12850098
596,Cafe Chantilly,Amsterdam,Cafe,597.0,5.0,1,42.0,d10136222
597,Carnivore Bar-Smoke BBQ,Amsterdam,American,598.0,5.0,2,54.0,d12443830
649,De Biertuin Prinsengracht,Amsterdam,Bar,650.0,5.0,2,24.0,d12176036
660,Barasti,Amsterdam,Dutch,661.0,5.0,2,26.0,d12600385


In [476]:
Ratings_v5 = Ratings_v4[Ratings_v4['Name'].isin(restaurants)]
#Ratings_v5 = Ratings_v4[Ratings_v4['City'].isin(list(R.index))]
#R = Ratings_v4.groupby(['City']).agg(sum).sort_values('Number_of_Reviews', ascending = False).head(n= 10)
Ratings_v5

,Name,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,ID_TA
127,Arles,Amsterdam,French,128.0,5.0,2,87.0,d11671406
141,Bistrot des Alpes,Amsterdam,French,142.0,5.0,2,136.0,d7735788
233,Benny’s Chicken,Amsterdam,Dutch,234.0,5.0,1,88.0,d10522612
359,Cuddle,Amsterdam,Bar,360.0,5.0,1,116.0,d11949427
421,Broer,Amsterdam,Dutch,422.0,5.0,2,50.0,d10525937
475,Chez Lorraine,Amsterdam,French,476.0,5.0,2,46.0,d12850098
596,Cafe Chantilly,Amsterdam,Cafe,597.0,5.0,1,42.0,d10136222
597,Carnivore Bar-Smoke BBQ,Amsterdam,American,598.0,5.0,2,54.0,d12443830
649,De Biertuin Prinsengracht,Amsterdam,Bar,650.0,5.0,2,24.0,d12176036
660,Barasti,Amsterdam,Dutch,661.0,5.0,2,26.0,d12600385


In [477]:
Ratings_v5.loc[(Ratings_v5['City'] == 'Amsterdam') & (Ratings_v5['Price_Range'] == 1), ['Name', 'Cuisine_Style']]

,Name,Cuisine_Style
233,Benny’s Chicken,Dutch
359,Cuddle,Bar
596,Cafe Chantilly,Cafe
885,Berry,Cafe
1073,Da Magda,Italian
1199,Bar Joost,Bar
1305,BROODJESZAAK t KUYLTJE,Dutch
1354,Croissanterie Friends,Cafe
1404,Arepa Shop,Vegetarian Friendly
1426,Anna's SpecialTeas,Cafe


In [478]:
#R1 = Ratings_v4.groupby(['City']).groupby(['Name'])['Number_of_Reviews'].agg(sum).sort_values(ascending = False)
#R1
#Ratings_v4.set_index(['City', 'Name'], inplace = True)

In [479]:
#Ratings_v4.loc[['Amsterdam', "De Silveren Spiegel"]]

In [480]:
#Ratings_v4.reset_index(drop= True)
Ratings_v5['Cuisine_Style']

127                    French
141                    French
233                     Dutch
359                       Bar
421                     Dutch
475                    French
596                      Cafe
597                  American
649                       Bar
660                     Dutch
707             International
840                    French
843             International
885                      Cafe
1073                  Italian
1095                  Italian
1199                      Bar
1250            Mediterranean
1273                  Chinese
1274                    Dutch
1305                    Dutch
1324                      Bar
1354                     Cafe
1360                    Dutch
1404      Vegetarian Friendly
1426                     Cafe
1433                 European
1450                     Cafe
1453                   French
1464                  Seafood
                 ...         
124433                  Asian
124473                    Bar
124490    

In [481]:
Ratings_v5['City'].unique()

array(['Amsterdam', 'Athens', 'Barcelona', 'Berlin', 'Bratislava',
       'Brussels', 'Budapest', 'Copenhagen', 'Dublin', 'Edinburgh',
       'Geneva', 'Hamburg', 'Helsinki', 'Krakow', 'Lisbon', 'Ljubljana',
       'London', 'Luxembourg', 'Lyon', 'Madrid', 'Milan', 'Munich',
       'Oporto', 'Oslo', 'Paris', 'Prague', 'Rome', 'Stockholm', 'Vienna',
       'Warsaw', 'Zurich'], dtype=object)

In [482]:
Ratings_v5 = Ratings_v5.drop(['Ranking', 'Number_of_Reviews', 'ID_TA'], axis = 1)

In [483]:
Ratings_v5.to_csv('restaurant_app/Restaurants_cleaned.csv', index= False)
#Ratings_v4

In [484]:
Ratings_v6 = pd.read_csv('restaurant_app/Restaurants_cleaned.csv')
#list(R.index)
Ratings_v6['Cuisine_Style'].values

array(['French', 'French', 'Dutch', ..., 'Swiss', 'Austrian', 'Bar'],
      dtype=object)

In [485]:
#Ratings_v5 = Ratings_v4[Ratings_v4['City'].isin(list(R.index))]
#Ratings_v5

In [124]:
#one_hot = MultiLabelBinarizer()
"""
Ratings_v5 = Ratings_v4.join(pd.DataFrame(one_hot.fit_transform(Ratings_v4.pop('Cuisine_Style')), columns = one_hot.classes_,
                                         index= Ratings_v4.index))  
Ratings_v5.head()

"""


"\nRatings_v5 = Ratings_v4.join(pd.DataFrame(one_hot.fit_transform(Ratings_v4.pop('Cuisine_Style')), columns = one_hot.classes_,\n                                         index= Ratings_v4.index))  \nRatings_v5.head()\n\n"

In [102]:
#one_hot.classes_
#Ratings_v4.join(Ratings_v5['Vegetarian Friendly'])
# Encode labels with value between 0 and n_classes-1
class MultiColumnLabelEncoder:
    def __init__(self, X, columns = None):
        self.df = deepcopy(X)
        self.columns = columns
    def fitransform(self):
        if not self.columns:            
            self.df.apply(LabelEncoder().fit_transform)
        else:
            for col in self.columns:
                #print(self.df[col])
                self.df[col] = LabelEncoder().fit_transform(self.df[col].astype(str))
        return self.df
#LE = MultiColumnLabelEncoder()

In [103]:
LE = MultiColumnLabelEncoder(Ratings_v5, ['Name', 'City', 'ID_TA'])
Ratings_v5 = LE.fitransform()

In [105]:
Ratings_v5 = Ratings_v5.iloc[:,0:7]
Ratings_v5 = Ratings_v5.rename(columns= {'Name':'Restaurants', 'ID_TA':'Users'})

In [106]:
Ratings_v5 

,Restaurants,City,Ranking,Rating,Price_Range,Number_of_Reviews,Users
0,25002,0,1.0,5.0,2,136.0,6154
1,10213,0,2.0,4.5,4,812.0,34200
2,20250,0,3.0,4.5,4,567.0,34551
3,42306,0,4.0,5.0,4,564.0,8852
4,23400,0,5.0,4.5,4,316.0,33892
5,8746,0,6.0,4.5,4,745.0,34524
6,43382,0,7.0,4.5,2,1455.0,705
7,4530,0,8.0,4.5,4,675.0,34593
8,38137,0,9.0,4.5,4,923.0,34573
9,34250,0,10.0,4.5,1,450.0,14646


In [19]:
train, test = train_test_split(Ratings_v5, test_size = 0.2)


In [20]:
# 'Restaurants', 'Users'
n_users, n_restaurant = len(Ratings_v5['Users'].unique()), len(Ratings_v5['Restaurants'].unique())
n_latent_factors = 10 # why 8 - 3 main regions, 5 different cost ranges

In [21]:
restaurant_input = Input(shape=[1],name='Item')
restaurant_embedding = Embedding(n_restaurant, n_latent_factors, name='NonNegrestaurant-Embedding', embeddings_constraint=non_neg())(restaurant_input)
restaurant_vec = Flatten(name='Flattenrestaurant')(restaurant_embedding)

user_input = Input(shape=[1],name='User')
user_vec = Flatten(name='FlattenUsers')(Embedding(n_users, n_latent_factors,name='NonNegUser-Embedding',embeddings_constraint=non_neg())(user_input))

prod = dot([restaurant_vec, user_vec], axes=1, normalize=False) #Normalized: yes or no?
model = Model([user_input, restaurant_input], prod)
model.compile('adam', 'mean_squared_error')

Instructions for updating:
Colocations handled automatically by placer.


In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
NonNegrestaurant-Embedding (Emb (None, 1, 10)        678620      Item[0][0]                       
__________________________________________________________________________________________________
NonNegUser-Embedding (Embedding (None, 1, 10)        741350      User[0][0]                       
__________________________________________________________________________________________________
Flattenres

In [ ]:
#add validation data in this step 
history_nonneg = model.fit([train['Users'], train['Restaurants']], 
                           train['Rating'], 
                           epochs=100, verbose=0)

In [ ]:
y_hat = np.round(model.predict([test['Users'], test['Restaurants']]),0)
y_true = test['']
mean_absolute_error(y_true, y_hat)

In [ ]:
model.save('Restaurants_cf_nonneg.h5')


In [ ]:
model = load_model('Restaurants_cf_nonneg.h5')
res = model.get_layer('NonNegrestaurant-Embedding')
res_weights = res.get_weights()[0]
lens = np.linalg.norm(res_weights, axis = 1)
normalized = (res_weights.T / lens).T

In [ ]:
def similarity_out(index):
    dists = np.dot(normalized, normalized[index])
    closest = np.argsort(dists)[::-1]
    return dists

In [ ]:
res_dict = {}
with open("dictionary_res.json", "w+") as f:
    json.dump(res_dict, f)

res_dict